<a href="https://colab.research.google.com/github/feniix/kinemotion/blob/main/presentation/demos/api_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kinemotion Python API Demo

Interactive demonstration of Kinemotion's Python API for drop jump analysis.

## Setup

Make sure Kinemotion is installed:
```bash
pip install kinemotion
```


In [ ]:
!pip install kinemotion

In [ ]:
# Import Kinemotion
import json
from pathlib import Path

from kinemotion import process_dropjump_video


## Demo 1: Single Video Analysis

Process a single drop jump video and extract metrics.


In [ ]:
# Process a drop jump video
video_path = "sample_data/IMG_5809.MOV"

print(f"Processing: {video_path}")
print("=" * 60)

metrics = process_dropjump_video(video_path, quality="balanced")

print("\n✅ Analysis Complete!")
print("\n📊 Key Metrics:")

# Handle potential None values
gct_ms = metrics.ground_contact_time * 1000 if metrics.ground_contact_time else 0
ft_ms = metrics.flight_time * 1000 if metrics.flight_time else 0
jh_m = metrics.jump_height if metrics.jump_height else 0

print(f"  Ground Contact Time: {gct_ms:.0f} ms")
print(f"  Flight Time: {ft_ms:.0f} ms")
print(f"  Jump Height: {jh_m:.3f} m")

# Calculate RSI manually (jump height / ground contact time)
gct = metrics.ground_contact_time if metrics.ground_contact_time else 0
rsi = metrics.jump_height / gct if gct and metrics.jump_height else 0
print(f"  Reactive Strength Index: {rsi:.2f}")

## Demo 2: Access All Metrics

Explore the complete metrics dictionary.


In [ ]:
# Convert metrics to dictionary for easy viewing
metrics_dict = metrics.to_dict()

print("📋 Complete Metrics:")
print(json.dumps(metrics_dict, indent=2))


## Demo 3: Custom Analysis Workflow

Analyze multiple jumps from the same athlete.

In [ ]:
# Process multiple jumps from the same athlete
from typing import Any

jump_videos = [
    "IMG_5809.MOV",
    "IMG_5813.MOV",
    "IMG_5820.MOV",
]

results: list[dict[str, Any]] = []

print("🏃 Processing athlete jumps...\n")

for jump_video in jump_videos:
    video_path = Path("sample_data") / jump_video

    if not video_path.exists():
        print(f"⚠️  Skipping {jump_video} (not found)")
        continue

    try:
        metrics = process_dropjump_video(str(video_path))
        # Calculate RSI manually
        gct = metrics.ground_contact_time if metrics.ground_contact_time else 0
        jh = metrics.jump_height if metrics.jump_height else 0
        rsi = jh / gct if (gct and jh) else 0
        results.append({
            "jump": jump_video,
            "gct_ms": metrics.ground_contact_time * 1000 if metrics.ground_contact_time else 0,
            "flight_time_ms": metrics.flight_time * 1000 if metrics.flight_time else 0,
            "rsi": rsi,
            "jump_height_m": jh
        })
        print(f"✅ {jump_video}: RSI = {rsi:.2f}")
    except Exception as e:
        print(f"❌ {jump_video}: {e}")

print(f"\n✅ Processed {len(results)} jumps")

## Demo 4: Data Analysis

Analyze the results and identify the best jump performance.

In [ ]:
# Sort by RSI (higher is better)
sorted_results = sorted(results, key=lambda x: x["rsi"], reverse=True)

print("🏆 Jump Performance Rankings (by RSI):\n")
for i, result in enumerate(sorted_results, 1):
    jump: str = result["jump"]
    rsi: float = result["rsi"]
    gct_ms: float = result["gct_ms"]
    jump_height_m: float = result["jump_height_m"]

    print(f"{i}. {jump}")
    print(f"   RSI: {rsi:.2f}")
    print(f"   GCT: {gct_ms:.0f}ms")
    print(f"   Jump Height: {jump_height_m:.3f}m")
    print()

## Demo 5: Export Results

Save results for further analysis in Excel, R, or SPSS.


In [ ]:
import csv

# Export to CSV
output_file = "jump_results.csv"

with open(output_file, 'w', newline='') as f:
    if results:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)

print(f"💾 Results exported to: {output_file}")
print("📊 Ready for statistical analysis!")

## Key Takeaways

✅ **Simple API** - Just import and call  
✅ **Flexible** - Build custom workflows  
✅ **Extensible** - Integrate with existing systems  
✅ **Export-friendly** - CSV, JSON, or custom formats  

## Questions?

- GitHub: https://github.com/feniix/kinemotion
- PyPI: https://pypi.org/project/kinemotion
- Docs: Full API reference in repository
